<a href="https://colab.research.google.com/github/karinayanh/Phyton_Assignments/blob/main/PC_Tech_product_mix_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PC Tech Product Mix Optimization

## The PC Tech Company: Background

**The PC Tech Company** assembles and then tests two models of computers: **Basic** and **XP**.  
- PC Tech Company wants to know how many of each model it should produce for the coming month to **maximize its net profit**, given constraints on labor hours and sales limits.
- **No inventory** is carried over from the previous month, and PC Tech does not want to hold any inventory after this month.
- **Available labor hours:** 10,000 hours for assembly and 3,000 hours for testing  
- **No more than** 600 Basics and 1,200 XPs can be sold this month  

---

## Input Data Summary

| Category                | Basic        | XP          | Notes                  |
|--------------------------|--------------|-------------|------------------------|
| **Labor hours required**     | 5 (assembly) + 1 (testing) | 6 (assembly) + 2 (testing) | per computer |
| **Labor cost per hour**      | \$11 (assembly) | \$15 (testing) | applies to both models |
| **Component cost**           | \$150         | \$225        | per computer |
| **Selling price**            | \$300         | \$450        | per computer |
| **Available labor hours**    | 10,000 (assembly) | 3,000 (testing) | total available |
| **Maximum sales**            | 600           | 1,200        | units |


---

## Mathematical Model

### Step 1: Identify the Decision Variables
Let:

**$x_1$**: Number of Basic computers produced

**$x_2$**: Number of XP computers produced

Both variables must be integers.


### Step 2: Figure out Objective Function

Each computer earns profit = selling price − component cost − labor costs.
- Profit per Basic = 300 - 150 - 5 * 11 - 1 * 15 = \$80  
- Profit per XP = 450 - 225 - 6 * 11 - 2* 15 = \$129  

$$
\text{Maximize } 80x_1 + 129x_2
$$

### Step 3: Impose the Constraints
| Constraint | Equation | Description |
|-------------|-----------------------|-------------|
| Assembly hours | `5x₁ + 6x₂ ≤ 10000` | Assembly time limit |
| Testing hours | `x₁ + 2x₂ ≤ 3000` | Testing time limit |
| Sales limit (Basic) | `x₁ ≤ 600` | Sales limit |
| Sales limit (XP) | `x₂ ≤ 1200` | Sales limit |
| Nonnegativity | `x₁, x₂ ≥ 0` | Cannot produce negative units |

---

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# 0. Read data from Excel
file_path = "/content/Product Mix Problem.xlsx"
df = pd.read_excel(file_path, sheet_name=0, header=None) # Read entire sheet

# Read the coefficients, and then convert them to floats, and store them as a NumPy array.
c = df.iloc[1, 2:4].astype(float).to_numpy()   # Objective coefficients  C2:D2 (row index 1, columns 2 and 3)
A = df.iloc[3:9, 2:4].astype(float).to_numpy() # Constraint matrix  C4:D9 (row indices 3..8, columns 2..3)
b = df.iloc[3:9, 5].astype(float).to_numpy()   # Right-Hand Side Vector  F4:F9 (row indices 3..8, column 5)

m, n = A.shape
print(f"(m = {m} constraints, n = {n} variables)\n")
print(A)
print(b)

(m = 6 constraints, n = 2 variables)

[[5. 6.]
 [1. 2.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[10000.  3000.   600.  1200.     0.     0.]


In [ ]:
# Example of a single constraint written explicitly (e.g., 5*x1 + 6*x2 <= 10000)
print(f" {A[0, 0] }*x1+{A[0, 1]}*x2 <= {b[0]}")

# Using a loop:
# With a single for-loop, we can generate all constraints from the matrix A and RHS vector b.

 5.0*x1+6.0*x2 <= 10000.0



## Python Implementation (Using Pyomo + HiGHS)

The modeling process follows exactly the same steps as the Beverage Planning Optimization

Difference: the decision variables are integers here

- **`pyo.ConcreteModel()`**  

- <span style="color:red"> **`pyo.Var(domain=pyo.NonNegativeIntegers)`** (integer, ≥0)</span>  

- **`pyo.Objective(expr=..., sense=pyo.maximize)`**

- **`pyo.ConstraintList()`** <span style="color:red"> (use ConstraintList and add constraints one by one)</span>

- **`pyo.SolverFactory("appsi_highs")`**

- **`solver.solve(model)`**   

- **`pyo.value(model.x1)`**

In [ ]:
# 1. Create the model
model = pyo.ConcreteModel(name="PC_Tech_Product_Mix")

# Define decision variables: (integer, ≥0)
model.x1 = pyo.Var(domain=pyo.NonNegativeIntegers)  # number of Basic computers
model.x2 = pyo.Var(domain=pyo.NonNegativeIntegers)  # number of XP computers

# Set the objective function: Maximize total net profit
model.obj = pyo.Objective(expr = c[0] * model.x1 + c[1] * model.x2, sense = pyo.maximize)

# Add Constraints
model.cons = pyo.ConstraintList() # use ConstraintList and add rows one by one (Ax <= b)
for i in range(m):
    if i<4:
        model.cons.add(A[i, 0] * model.x1 + A[i, 1] * model.x2 <= b[i]) # Labor hours limits and Sales limit
    else:
        model.cons.add(A[i, 0] * model.x1 + A[i, 1] * model.x2 >= b[i]) # Nonnegativity


# 2. Solve using APPsi–HiGHS
solver = pyo.SolverFactory("appsi_highs")
result = solver.solve(model)

# 3. Display results
print("\033[1mOptimal Integer Solution\033[0m")
print(f"Solver Status      : {result.solver.status}")
print(f"Termination Reason : {result.solver.termination_condition}\n")

x1 = int(pyo.value(model.x1))
x2 = int(pyo.value(model.x2))
profit = pyo.value(model.obj)

print(f"\033[1mOptimal Production Plan:\033[0m")
print(f"  x1 (Basic computers) : {x1} units") # The company produces 560 Basic computers
print(f"  x2 (XP computers)    : {x2} units") # The company produces 1200 XP computers
print(f"Total Profit: ${profit:,.2f}") # The company earns net profit $199,600

Optimal Integer Solution
Solver Status      : ok
Termination Reason : optimal

Optimal Production Plan:
  x1 (Basic computers) : 560 units
  x2 (XP computers)    : 1200 units
Total Profit: $199,600.00


In [ ]:
# Calculate total resource use
assembly_used = A[0,0]*x1 + A[0,1]*x2 # The company use 10000 assembly hours
testing_used  = A[1,0]*x1 + A[1,1]*x2 # The company use 2960 testing hours
print(f"Assembly hours used: {assembly_used} / {b[0]}")
print(f"Testing  hours used: {testing_used} / {b[1]}")

Assembly hours used: 10000.0 / 10000.0
Testing  hours used: 2960.0 / 3000.0
